In [1]:
import requests
import json
import time
import pandas as pd
from IPython.display import clear_output

loop_count = 250
beer_count = 0

beer_df = pd.DataFrame(columns=['id','name','abv','ph','ibu', 'first_brewed', 'hops'])

for x in range(220, loop_count+1):
    #Use IPython custome clear output to minimise outputs
    clear_output(wait=True)
    
    #Get JSON using URL Request
    url_api = 'https://api.punkapi.com/v2/beers/' + str(x)
    response = requests.get(url=url_api, headers = {'Content-Type': 'application/json', 'Accept': 'application/json'})
    
    #If error, API will return differently structured JSON rather than a HTTP error
    try:
        #Store JSON, grabbing first element (due to JSON structure from API)
        json_beer = json.loads(response.text)[0]
    except:
        #Error - so skip (but show ID with error)
        print("Grabbing Punk beer data. Current progress:",x,"/",loop_count,". ",beer_count," beer(s) discovered so far! No beer with ID: ",x)
        continue
        
    #Assign beer data into Dictionary for later analysis
    beer_list = {}
    
    #Create lists for hops and yeast
    hops_list = []
    yeast_list = []
    
    #Store beer info from JSON
    beer_list['id'] = json_beer['id']
    beer_list['name'] = json_beer['name']
    beer_list['abv'] = json_beer['abv']
    beer_list['ph'] = json_beer['ph']
    beer_list['ibu'] = json_beer['ibu'] 
    beer_list['first_brewed'] = json_beer['first_brewed']
    
    #Use a List comprehension to get all hops types per beer
    hops_list = [hop['name'] for hop in json_beer['ingredients']['hops']]
    
    #Remove hop duplicates (by creating a set) and put into array
    hops_list = list(set(hops_list))
    beer_list['hops'] = hops_list
    
    #Append to dataframe
    temp_df  = pd.DataFrame([beer_list], columns=beer_list.keys())
    beer_df = beer_df.append(temp_df)
    
    #Add beer count
    beer_count  += 1
    
    #Print progress
    print("Grabbing Punk beer data. Current progress:",x,"/",loop_count,". ",beer_count," beer(s) discovered so far!")
    
    #Wait fixed time to prevent API throttling / restricting usage
    time.sleep(.75)

#Reset index of Dataframe (ensuring that old index doesn't get added as a column)
beer_df = beer_df.reset_index(drop=True)

Grabbing Punk beer data. Current progress: 250 / 250 .  15  beer(s) discovered so far! No beer with ID:  250


In [14]:
print("Hypothesis 1: There is a relationship between the choice of hop when brewing and the IBU (International Bittering Units)")

import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

hops_df = pd.DataFrame(columns=['name','count'])

for index, row in beer_df.iterrows():
    for hop_name in row['hops']:
        print(hop_name)
        if hop_name not in hops_df.name:
            temp_df  = pd.DataFrame(np.array([hop_name, 1]), columns=hops_df.keys())
            hops_df = hops_df.append(temp_df)
        else:
            hop_index = hops_df.index[hops_df['name'] == hop_name].tolist()
            hops_df.at[hop_index, 'count'] += 1
            

Hypothesis 1: There is a relationship between the choice of hop when brewing and the IBU (International Bittering Units)
Simcoe


ValueError: Shape of passed values is (1, 2), indices imply (2, 2)